<a href="https://colab.research.google.com/github/mvish7/deep_learning_handson/blob/master/softmax%20regression%20from%20scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install mxnet==1.6.0b20190926
!pip install d2l

In [0]:
import sys
sys.path.insert(0, '..')

%matplotlib inline
import numpy as np
import d2l
import torch, torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.distributions import normal

creating data loader instance

In [0]:
batch_size = 256
trans = transforms.ToTensor()

train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

Random init of weights and bias

In [0]:
num_inputs = 784
num_outputs = 10

W = normal.Normal(loc = 0, scale = 0.01).sample((num_inputs, num_outputs))
b = torch.zeros(num_outputs)

W.requires_grad_(True)
b.requires_grad_(True)

implementing softmax 

In [0]:
def softmax(X):
    X_exp = torch.exp(X)
    norm_const = torch.sum(X_exp, dim=1, keepdim=True)
    return X_exp / norm_const

checking of softmax fun works or not

In [27]:
X = normal.Normal(loc = 0, scale = 1).sample((2, 5))
X_prob = softmax(X)
X_prob, torch.sum(X_prob, dim=1)

(tensor([[0.0440, 0.0478, 0.3440, 0.3804, 0.1839],
         [0.4796, 0.0274, 0.0385, 0.2239, 0.2307]]), tensor([1.0000, 1.0000]))

model defination

In [0]:
def net(X):
    x = torch.matmul(X.reshape((-1, num_inputs)), W) + b
    return softmax(x)

defining cross entropy loss

In [0]:
def cross_entropy(y_hat, y):
    return -torch.gather(y_hat, 1, y.unsqueeze(dim=1)).log() #hats off to this, i was thinking of using loop :-p

fun for evaluating accuracy on test data -- taken from d2l

In [0]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).sum().item()
        n += y.size()[0]  # y.size()[0] = batch_size
    return acc_sum / n

training loop

In [0]:
num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params=None, lr=None, trainer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            l.backward()
            if trainer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                # This will be illustrated in the next section
                trainer.step(batch_size)
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.size()[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [0]:
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)